<a href="https://colab.research.google.com/github/HussamSelim/Transfer-learning-MNIST-CNN/blob/main/Transfer_learning_on_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D,Dropout,Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as k

In [23]:
num_classes=5
pool_size=2
kernel_size=3
epochs=5

In [24]:
if k.image_data_format=='channels_first':
  input_shape=(1,28,28)
else:
  input_shape=(28,28,1)

In [25]:
def model_train(model,train,test,num_classes):
  x_train=train[0].reshape((train[0].shape[0],)+input_shape)
  x_test=test[0].reshape((test[0].shape[0],)+input_shape)

  x_train=x_train.astype('float32')
  x_test=x_test.astype('float32')

  x_train /= 255
  x_test /= 255

  # convert class vectors to binary class matrices
  y_train=tf.keras.utils.to_categorical(train[1],num_classes)
  y_test=tf.keras.utils.to_categorical(test[1],num_classes)

  model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adadelta')

  model.fit(x_train,y_train,
            epochs=4,
            verbose=1,
            validation_data=(x_test,y_test)
            )
  
  score=model.evaluate(x_test,y_test,verbose=0)

  




In [26]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train_lt5=x_train[y_train<5]
y_train_lt5=y_train[y_train<5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]


x_train_gte5=x_train[y_train>=5]
y_train_gte5=y_train[y_train>=5] -5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] -5

In [27]:
feature_layers=[
                
                Conv2D(32,kernel_size,input_shape=input_shape),
                Activation('relu'),
                Conv2D(32,kernel_size),
                Activation('relu'),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.25),
                Flatten()
]

In [28]:
classification_layer=[
                      
                      Dense(128),
                      Activation('relu'),
                      Dropout(0.4),
                      Dense(num_classes),
                      Activation('softmax')

]

In [29]:
model=Sequential(feature_layers+classification_layer)

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_8 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_9 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)             

In [31]:
model_train(model,(x_train_lt5,y_train_lt5),(x_test_lt5,y_test_lt5),num_classes)

Epoch 1/4
957/957 [==============================] - 9s 9ms/step - loss: 1.5569 - accuracy: 0.3372 - val_loss: 1.4900 - val_accuracy: 0.6754
Epoch 2/4
957/957 [==============================] - 8s 8ms/step - loss: 1.4325 - accuracy: 0.6264 - val_loss: 1.3301 - val_accuracy: 0.8716
Epoch 3/4
957/957 [==============================] - 8s 8ms/step - loss: 1.2575 - accuracy: 0.7827 - val_loss: 1.1103 - val_accuracy: 0.9033
Epoch 4/4
957/957 [==============================] - 8s 8ms/step - loss: 1.0339 - accuracy: 0.8368 - val_loss: 0.8492 - val_accuracy: 0.9185


In [32]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

In [33]:
model_train(model,(x_train_gte5,y_train_gte5),(x_test_gte5,y_test_gte5),num_classes)

Epoch 1/4
919/919 [==============================] - 6s 6ms/step - loss: 1.5508 - accuracy: 0.3331 - val_loss: 1.4234 - val_accuracy: 0.4211
Epoch 2/4
919/919 [==============================] - 5s 5ms/step - loss: 1.3734 - accuracy: 0.4571 - val_loss: 1.2544 - val_accuracy: 0.5894
Epoch 3/4
919/919 [==============================] - 5s 5ms/step - loss: 1.2305 - accuracy: 0.5784 - val_loss: 1.1114 - val_accuracy: 0.7079
Epoch 4/4
919/919 [==============================] - 5s 6ms/step - loss: 1.1019 - accuracy: 0.6759 - val_loss: 0.9905 - val_accuracy: 0.7760
